In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens)

# The actual deposits.

v3.deposit(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens, 'bnt')

v3.trade(tkn_amt=2000, source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)

v3.trade(tkn_amt=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)

v3.trade(tkn_amt=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=0)


def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Bancor Vortex

In [2]:
v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=60022.384703,bnt=60023.599777,bnbnt=60000.000000,bnt=12.269231,bnt=0.000000,bnbnt=60000.000000
1,bnt=19055.238095 eth=21.000000,eth=102.000000,eth=101.000000,bneth=101.000000,,eth=0.000000,
2,bnt=20940.883461 wbtc=1.194287,wbtc=100.944287,wbtc=101.000450,bnwbtc=101.000000,,wbtc=0.000000,
3,bnt=20013.993917 link=3336.331433,link=10003.998100,link=10003.424242,bnlink=10001.000000,,link=0.000000,
4,,,,,,,


The function and goal of the Bancor Vortex remains unchanged; the value it has accrued from its share of the revenues is used to buy and burn vBNT. However, the manner in which it achieves this is significantly improved.

Previously, the Bancor Vortex was powered by a dedicated wallet that would fill with each of the TKN from which it had earned fees. Each of these tokens would then be swapped at a later time on their respective pools for BNT, and that BNT is used to buy and burn vBNT. Conceptually, this process is maintained in Bancor 3; however, as demonstrated above, the TKN fees are translated into BNT on a continual basis. Therefore, the burner funds are collected exclusively in BNT, which dramatically simplifies the overall process and improves the gas efficiency of performing the burn.

Since the BNT used to power the Vortex are already in the vault, the act of performing a swap is also simplified. Consider that the protocol is using BNT inside the vault, to purchase vBNT that is already inside the vault alongside it. Performing a trade between the two assets therefore does not involve any token transfers at all. Instead, the liquidity balances of the BNT/vBNT pool are simply updated as though a swap was performed. Therefore, when the Vortex is triggered, the balance on its ledger is reset to 0, a virtual trade is performed on the BNT/vBNT liquidity pool, and vBNT is burned directly from the vault. To demonstrate the process, a simplified view of the protocol is presented below, where three human users, and the Bancor Protocol itself, are the only participants in the system.

![](https://lh3.googleusercontent.com/yOjXVeIWV4jC-tIGMAi-RDWKrS3mQPMSQcDhM79xbnJXPYgYkcPIo7TDGLlTYrvkFbhmWjcmcmHYEywAF6EkGb1t4y8IbRL1ee0hyGtGyJxzm-CQ95CirBLSbGhEcshkikdG5G9L)

![](https://lh6.googleusercontent.com/nh_ztQVdqmj5k-c70a7soEds74ImrLoirTz5c7lJ674agNHwCsBfWTLcWnNzF_NQbBo_VsMu5a338l3JyaogF5rjnz4yhrJz7sZE4zdtbX5oTIwt3Hzok57ZODerBjQp7m8xr7Cn)

For this scenario, the staked amount of BNT is set high to reflect the assumption that there are other pools in the network other than vBNT; however, they aren't shown, because they are not an important component of this demonstration.

At this state, triggering the Vortex causes a virtual swap on the vBNT pool to occur. The equations that describe this process are presented in the above sections, and are repeated here for completeness. The change in the BNT and vBNT trading liquidity can be expressed as follows:

![](https://lh3.googleusercontent.com/kk9N2rYg_vT85LUHgA_RNrg04kHac4We8KvNBtV3_dw0K1q6LcAfJ7_NF3R5h1JEsuT5pn5S0Azi5vQ5PaFoHxMw3StkoADFpyKN6EQiXt1vON1gbbhfi5ayDraD2U8vE5FNbgCw)

![](https://lh3.googleusercontent.com/sg_YY1cyR6IsWBKfiUbj5v9rREz9A7PO8FmpK8uPov7SmyoYrK4H6FroNDZ7EOJ9LcC0IOVggaLW4J3rVx1rwrEzl0Aqd55Ec_5d_cXQwNhjWlWNO73Dr3XJq_wBAoXvdC_i3gTf)

![](https://lh4.googleusercontent.com/SvZOi1YaY_Bw_oNVmqje8I5GK2qfhh_e-ZYJpFrFAuMv94vB56IfTqE_cQT5ZXH5V-zAMV0ZSoVGFBK6UQL13xmF_RGha6aWDGHtxtyain_XDaYcmqHOdOrp-JW04h2pJ6-IUzk0)

where a1† and a1 are the BNT trading liquidity balances of the pool after and before the trade, respectively, b1† and b1 are the vBNT trading liquidity balances of the pool after and before the trade, respectively, d1 is the pool fee (e.g. 0.01, or 1%) and e is the Vortex rate. Unlike standard trades, a Vortex burner does not trigger its own fee collection. Therefore, the Vortex rate, e, is set to zero during this swap. The number of BNT tokens being sent into the vault is x, and is equal to the balance on the Vortex ledger. The number of vBNT tokens removed from the trading liquidity, and burned from the vault is tknOut. The burner still causes fee accumulation to vBNT liquidity providers, denominated in vBNT, and is added to the staking ledger as normal:

![](https://lh5.googleusercontent.com/NUQMFTb0HxIOqjiTTDtdH6idb2J9MZ5yRQgWM-M-m64TXOkOfEnRRAHOeHVTGrqjC4N0eumSVWGuYOz_uDhJ81QpyHN_-u-IzQwxPf7PKdfp_oG6fOzz0_X7tejwut7RSn5Jb9oV)

For this example, assume the swap fee on the vBNT pool is 1%. The virtual swap of 123 BNT (i.e. the balance on the Vortex ledger) returns 114.715026 vBNT; however, recall that this trade is happening entirely within the vault. Therefore, ignoring the effect of the burn, the vault balances of BNT and vBNT remain completely unchanged - it is only the spot price reported by the pool that changes; there is no token transfer to process. The vBNT amount that is returned by this function represents the burn quantity, and so the vault balance of vBNT will be reduced by precisely this quantity after the virtual swap is completed. A fee of 1.158738 vBNT is added to the staked balance as normal.

![](https://lh5.googleusercontent.com/IeE2uFuUaDYe967uv_FOWankU0RrW4ilp7yBrH0ONLHgbm7nkKkEMMy6twzsMQlRM5BEgsd0Eg_5N_CYzbRj4IZ4is0NnuXff9ZylMV4l1O52X1NZN3y4O1i0s5ffo3rFi8-JT4z)

![](https://lh6.googleusercontent.com/-OxMsn7grQ3QaUy7FMZu8Z9v9GLRH0pzUIY8VP4nSDNkUpIUSS4oqf3g8olfQc5gT72TrJhLm_ytkF-k9rC8u93LeDCpB_SKHg4JcEq1-JbJVJAb83MFNLDi1kvVua3OCm1aoIpi)